In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

pd.set_option('display.float_format', lambda x: '%.2f' % x)

# 1. ЗАГРУЗКА ДАННЫХ

orders = pd.read_csv('olist_orders_dataset.csv')
order_items = pd.read_csv('olist_order_items_dataset.csv')
customers = pd.read_csv('olist_customers_dataset.csv')
products = pd.read_csv('olist_products_dataset.csv')
payments = pd.read_csv('olist_order_payments_dataset.csv')
category = pd.read_csv('product_category_name_translation.csv')
sellers = pd.read_csv('olist_sellers_dataset.csv')

In [2]:
# 2. ПРИВЕДЕНИЕ ДАТ К DATETIME

date_cols = [
    'order_purchase_timestamp',
    'order_approved_at',
    'order_delivered_carrier_date',
    'order_delivered_customer_date',
    'order_estimated_delivery_date'
]

for col in date_cols:
    orders[col] = pd.to_datetime(orders[col], errors='coerce')

In [3]:
# 3. ПРОВЕРКА ПРОПУСКОВ

print("Пропуски orders:")
print(orders.isnull().sum(), "\n")

print("Пропуски products:")
print(products.isnull().sum(), "\n")

print("Пропуски payments:")
print(payments.isnull().sum(), "\n")

Пропуски orders:
order_id                            0
customer_id                         0
order_status                        0
order_purchase_timestamp            0
order_approved_at                 160
order_delivered_carrier_date     1783
order_delivered_customer_date    2965
order_estimated_delivery_date       0
dtype: int64 

Пропуски products:
product_id                      0
product_category_name         276
product_name_lenght           277
product_description_lenght    277
product_photos_qty            277
product_weight_g                2
product_length_cm               2
product_height_cm               2
product_width_cm                2
dtype: int64 

Пропуски payments:
order_id                0
payment_sequential      0
payment_type            0
payment_installments    0
payment_value           0
dtype: int64 



In [4]:
# 4. ЧИСТКА PRODUCTS

products_clean = products.copy()
products_clean['product_category_name'] = products_clean['product_category_name'].fillna('unknown')

# ДОБАВИЛ ПРОПУЩЕННОЕ: перевод категорий
products_clean = products_clean.merge(
    category, on='product_category_name', how='left'
)

In [5]:
# 5. ЧИСТКА ORDERS

orders_clean = orders.copy()

orders_clean['is_delivered'] = orders_clean['order_delivered_customer_date'].notna().astype(int)
orders_clean['is_canceled'] = (orders_clean['order_status'] == 'canceled').astype(int)

orders_clean['is_in_transit'] = (
    (orders_clean['is_delivered'] == 0) &
    (orders_clean['is_canceled'] == 0)
).astype(int)

In [6]:
# 6. СОЗДАНИЕ ИТОГОВОГО DF

payments_sum = payments.groupby('order_id', as_index=False)['payment_value'].sum()

df = (
    orders_clean
      .merge(order_items, on='order_id')
      .merge(customers, on='customer_id')
      .merge(products_clean[['product_id', 'product_category_name', 'product_category_name_english']],
             on='product_id', how='left')
      .merge(payments_sum, on='order_id', how='left')
)

In [7]:
# 7. НОВЫЕ ПОЛЯ

df['quantity'] = 1
df['revenue'] = df['price'] * df['quantity']
df['gmv'] = df['price'] + df['freight_value']

df['order_date'] = df['order_purchase_timestamp'].dt.date
df['order_month'] = df['order_purchase_timestamp'].dt.to_period('M')
df['order_weekday'] = df['order_purchase_timestamp'].dt.day_name()

df['year'] = df['order_purchase_timestamp'].dt.year
df['month'] = df['order_purchase_timestamp'].dt.month

In [8]:
# 8. ПОИСК ВЫБРОСОВ

def detect_outliers_iqr(df, column):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    low = Q1 - 1.5 * IQR
    high = Q3 + 1.5 * IQR
    outliers = df[(df[column] < low) | (df[column] > high)]
    return outliers, low, high


outliers_rev, low_rev, high_rev = detect_outliers_iqr(df, 'revenue')

print(f'Границы revenue (IQR): {low_rev:.2f} – {high_rev:.2f}')
print(f'Количество выбросов: {len(outliers_rev)}')
print(f'Доля выбросов: {len(outliers_rev) / len(df):.2%}')


Границы revenue (IQR): -102.60 – 277.40
Количество выбросов: 8427
Доля выбросов: 7.48%


In [9]:
# 9. RFM

snapshot_date = df['order_purchase_timestamp'].max()

rfm = df.groupby('customer_unique_id').agg({
    'order_purchase_timestamp': lambda x: (snapshot_date - x.max()).days,
    'order_id': 'nunique',
    'revenue': 'sum'
}).reset_index()

rfm.columns = ['customer_id', 'recency', 'frequency', 'monetary']

In [10]:
# 10. АГРЕГАТЫ ДЛЯ BI

# AOV по месяцам
aov_by_month = (
    df.groupby(['year', 'month'])
      .agg(
          revenue=('revenue', 'sum'),
          orders=('order_id', 'nunique')
      )
      .reset_index()
)
aov_by_month['AOV'] = aov_by_month['revenue'] / aov_by_month['orders']

In [11]:
# AOV по категориям
aov_by_category = (
    df.groupby('product_category_name')
      .agg(
          revenue=('revenue', 'sum'),
          orders=('order_id', 'nunique')
      )
      .reset_index()
)
aov_by_category['AOV'] = aov_by_category['revenue'] / aov_by_category['orders']

In [12]:
# Repeat Rate
orders_per_customer = df.groupby('customer_unique_id')['order_id'].nunique()
repeat_rate = (orders_per_customer > 1).mean()

repeat_rate

np.float64(0.030528191154894153)

In [13]:
# Выручка по месяцам
revenue_by_month = (
    df.groupby(df['order_purchase_timestamp'].dt.to_period("M"))['revenue']
      .sum()
      .reset_index()
)

In [14]:
# Кол–во заказов по месяцам
orders_by_month = (
    df.groupby(df['order_purchase_timestamp'].dt.to_period("M"))['order_id']
      .nunique()
      .reset_index()
)

In [15]:
# 11. ЭКСПОРТ ДАТАСЕТОВ ДЛЯ BI

df.to_csv("df_facts.csv", index=False)
rfm.to_csv("df_rfm.csv", index=False)
aov_by_month.to_csv("df_aov_month.csv", index=False)
aov_by_category.to_csv("df_aov_category.csv", index=False)
revenue_by_month.to_csv("df_revenue_month.csv", index=False)
orders_by_month.to_csv("df_orders_month.csv", index=False)